In [ ]:
import numpy             as np
import matplotlib.pyplot as plt
import scipy.sparse      as sparse
import scipy.special     as sp

import hylife.utilitis_FEEC.bsplines             as bsp
import hylife.utilitis_FEEC.evaluation           as eva
import hylife.geometry.mappings_analytical       as mapping
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D

In [ ]:
Nel       = [16, 4, 4]              # mesh generation on logical domain
bc        = [True, True, True]      # boundary conditions (True: periodic, False: else)
p         = [3, 2, 2]               # spline degrees  


el_b      = [np.linspace(0., 1., Nel + 1) for Nel in Nel]                      # element boundaries
delta     = [1/Nel for Nel in Nel]                                             # element sizes
T         = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]  # knot vectors (for N functions)
t         = [T[1:-1] for T in T]                                               # reduced knot vectors (for D function)
NbaseN    = [Nel + p - bc*p for Nel, p, bc in zip(Nel, p, bc)]                 # number of basis functions (N functions)
NbaseD    = [NbaseN - (1 - bc) for NbaseN, bc in zip(NbaseN, bc)]              #number of basis functions (D functions)

kind_map = 1               #slab geometry
L_map    = [2., 1., 1.] #length of the simulation box 

args_map = kind_map, L_map #putting it together

In [ ]:
import hylife.utilitis_FEEC.projectors_global    as proj_glob
import hylife.utilitis_FEEC.projectors_local     as proj_loc
import hylife.utilitis_FEEC.derivatives          as der

proj_glob = proj_glob.projectors_3d(T, p, bc)
proj_loc  = proj_loc.projectors_local_3d(T, p, bc)

proj_glob.assemble_V3()
proj_glob.assemble_V2()
proj_glob.assemble_V1()
derivatives = der.discrete_derivatives(T, p, bc)
DIV  = derivatives.DIV_3d()

In [ ]:
rho_ini_phys = lambda x, y, z : np.sin(2*np.pi*(x/L_map[0]))

In [ ]:
vx_ini_phys = lambda x, y, z : 1.
vy_ini_phys = lambda x, y, z : 0.
vz_ini_phys = lambda x, y, z : 0.

In [ ]:
def rho_ini(xi1_arr, xi2_arr, xi3_arr):
    n1 = xi1_arr.shape[0]
    n2 = xi2_arr.shape[0]
    n3 = xi3_arr.shape[0]
    rho = np.empty((n1, n2, n3), dtype=float) # Returns numpy array of shape [n1,n2,n3] same fct as np.zeros
    for i1 in range(n1):
        for i2 in range(n2):
            for i3 in range(n3):
                idx = i1, i2, i3
                xi1 = xi1_arr[i1]
                xi2 = xi2_arr[i2]
                xi3 = xi3_arr[i3]
                x = mapping.f(xi1, xi2, xi3, *args_map, 1) #Analytic Mapping to slab Geometry
                y = mapping.f(xi1, xi2, xi3, *args_map, 2)
                z = mapping.f(xi1, xi2, xi3, *args_map, 3)
                rho[idx] = rho_ini_phys(x, y, z) * mapping.det_df(xi1, xi2, xi3, *args_map)
    return rho

In [ ]:
# input for projector V2: one point value and two 1d arrays
def v1_ini(xi1, xi2_arr, xi3_arr):
    n2 = xi2_arr.shape[0]
    n3 = xi3_arr.shape[0]
    v1 = np.empty((n2, n3), dtype=float)
    for i2 in range(n2):
        for i3 in range(n3):
            idx = i2, i3
            
            xi2 = xi2_arr[i2]
            xi3 = xi3_arr[i3]

            x = mapping.f(xi1, xi2, xi3, *args_map, 1)
            y = mapping.f(xi1, xi2, xi3, *args_map, 2)
            z = mapping.f(xi1, xi2, xi3, *args_map, 3)

            df_inv_11 = mapping.df_inv(xi1, xi2, xi3, *args_map, 11)
            df_inv_21 = mapping.df_inv(xi1, xi2, xi3, *args_map, 21)
            df_inv_31 = mapping.df_inv(xi1, xi2, xi3, *args_map, 31)

            v1[idx] = (  vx_ini_phys(x, y, z) * df_inv_11 
                       + vy_ini_phys(x, y, z) * df_inv_21 
                       + vz_ini_phys(x, y, z) * df_inv_31)
    return v1

In [ ]:
#def v1_pr(xi1_arr, xi2_arr, xi3_arr):
#    n1= xi1_arr.shape[0] #xi1_arr Logical Coordinates (?)
#    n2= xi2_arr.shape[0]
#    n3= xi3_arr.shape[0]
#    print(n2)
#    v1 = [] # Returns numpy array of shape [n1,n2,n3] same fct as np.zeros
#    for i1 in range(n1):
#        for i2 in range(n2):
#            for i3 in range(n3):
#                idx = i1, i2, i3
#                xi1 = xi1_arr[idx]
#                xi2 = xi2_arr[idx]
#                xi3 = xi3_arr[idx]
#                x = mapping.f(xi1, xi2, xi3, *args_map, 1) #Analytic Mapping to slab Geometry
#                y = mapping.f(xi1, xi2, xi3, *args_map, 2)
#                z = mapping.f(xi1, xi2, xi3, *args_map, 3)
#                v1.append(1)
#    return v1

In [ ]:
# input for projector V2: one point value and two 1d arrays
def v1_times_rho(xi1, xi2_arr, xi3_arr):
    global rho_coeff
    n2 = xi2_arr.shape[0]
    n3 = xi3_arr.shape[0]
    fun = np.empty((n2, n3), dtype=float)
    
    
    rho_h = eva.FEM_field_V3_3d(rho_coeff, [np.array([xi1]), xi2_arr, xi3_arr], T, p, bc)

    fun = rho_h.reshape(n2, n3) * v1_ini(xi1, xi2_arr, xi3_arr)
    
    return fun

In [ ]:
def v2_times_rho(xi1_arr, xi2, xi3_arr):
    global rho_coeff
    #global v_coeff  # [v1_coeff,v2_coeff,v3_coeff]
    n1 = xi1_arr.shape[0]
    n3 = xi3_arr.shape[0]
    fun = np.empty((n1, n3), dtype=float)
    
    rho_h = eva.FEM_field_V3_3d(rho_coeff, [xi1_arr, np.array([xi2]), xi3_arr], T, p, bc)
    #[v1_h, v2_h, v3_h] = eva.FEM_field_V1_3d(v_coeff, [xi1_arr1d, np.array([xi2]), xi3_arr1d], T, p, bc)
    fun = rho_h.reshape(n1, n3)*0. #* v2_h.reshape(n1, n3)
    return fun

In [ ]:
def v3_times_rho(xi1_arr, xi2_arr, xi3):
    global rho_coeff
    n1 = xi1_arr.shape[0]
    n2 = xi2_arr.shape[0]
    fun = np.empty((n1, n2), dtype=float)
    
    # Third coordinate xi3 is the one that the function (rho*v)3 depends on
    rho_h = eva.FEM_field_V3_3d(rho_coeff, [xi1_arr, xi2_arr, np.array([xi3])], T, p, bc)   
    fun = rho_h.reshape(n1, n2) * 0.
    
    return fun

#### $\rho_0$ Initialization and Visualization

In [ ]:
rho_coeff_init = proj_glob.PI_3(rho_ini)


In [ ]:
#v_coeff= proj_glob.PI_1([v1_pr,v1_pr,v1_pr])

In [ ]:
# prepare visualization
np1=100
np2=100
np3=1
xi1=np.linspace(0,1,np1)
xi2=np.linspace(0,1,np2)
xi3=np.array([0])

#x_m,y_m,z_m = push_forward_meshgrid([xi1,xi2,xi3]) 
x_m=np.empty((np1,np2,np3))
y_m=np.empty((np1,np2,np3))
z_m=np.empty((np1,np2,np3))
det=np.empty((np1,np2,np3))

for i1 in range(np1):
    for i2 in range(np2):
        for i3 in range(np3):
            idx=i1,i2,i3
            x_m[idx]= mapping.f(xi1[i1],xi2[i2],xi3[i3], *args_map, 1)
            y_m[idx]= mapping.f(xi1[i1],xi2[i2],xi3[i3], *args_map, 2)
            z_m[idx]= mapping.f(xi1[i1],xi2[i2],xi3[i3], *args_map, 2)
            
            det[idx] = mapping.det_df(xi1[i1],xi2[i2],xi3[i3], *args_map)
            

In [ ]:
%matplotlib notebook         
#rho_phys_0_plot =push_forward_V3(rho_0_plot,[xi1,xi2,xi3]) #divide by jacobian determinant

rho_0_plot = eva.FEM_field_V3_3d(rho_coeff_init, [xi1,xi2,xi3], T, p, bc) /det

    
#x_m, y_m=np.meshgrid(x,y)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x_m.reshape(np1,np2),y_m.reshape(np1,np2),rho_0_plot.reshape(np1,np2))
rho_ex_0=np.empty((np1,np2,np3))
for i1 in range(np1):
    for i2 in range(np2):
        for i3 in range(np3):
            idx=i1,i2,i3
            rho_ex_0[idx] = rho_ini_phys(x_m[idx],y_m[idx],z_m[idx])
          
ax.plot_surface(x_m.reshape(np1,np2),y_m.reshape(np1,np2),rho_ex_0.reshape(np1,np2))

In [ ]:
def eva_rho_t():
    global rho_coeff
    rhov1_coeff, rhov2_coeff, rhov3_coeff = proj_glob.PI_2([v1_times_rho, v2_times_rho, v3_times_rho])
    rho_t = -(DIV.dot(np.concatenate((rhov1_coeff.flatten(), rhov2_coeff.flatten(), rhov3_coeff.flatten())))).reshape(NbaseD[0], NbaseD[1], NbaseD[2]) 
    return rho_t

In [ ]:
#Explicit Euler Integration
rho_coeff=rho_coeff_init
dt=0.01
ntsteps=5
for it in range(ntsteps):
    rho_t=eva_rho_t()
    rho_coeff=(rho_coeff+dt*rho_t)

In [ ]:
%matplotlib notebook         

rho_1_plot = eva.FEM_field_V3_3d(rho_coeff, [xi1,xi2,xi3], T, p, bc) /det

    
#x_m, y_m=np.meshgrid(x,y)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x_m.reshape(np1,np2),y_m.reshape(np1,np2),rho_1_plot.reshape(np1,np2))

#exact density at time t=ntsteps*dt, translated with a constant velocity field vx,vy,vz
rho_ex_1=np.empty((np1,np2,np3))
for i1 in range(np1):
    for i2 in range(np2):
        for i3 in range(np3):
            idx=i1,i2,i3
            xt = x_m[idx] -ntsteps*dt*vx_ini_phys(x_m[idx],y_m[idx],z_m[idx])
            yt = y_m[idx] -ntsteps*dt*vy_ini_phys(x_m[idx],y_m[idx],z_m[idx])
            zt = z_m[idx] -ntsteps*dt*vz_ini_phys(x_m[idx],y_m[idx],z_m[idx])
            rho_ex_1[idx] = rho_ini_phys(xt,yt,zt)
          
ax.plot_surface(x_m.reshape(np1,np2),y_m.reshape(np1,np2),rho_ex_1.reshape(np1,np2))